In [1]:
# Import the functions used in this project
import awesome_functions as af
import decode_utils as du
from sklearn.externals import joblib

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost
from sklearn.metrics import log_loss
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
def fitNaiveBayesModel(X, y):
    return MultinomialNB(alpha=0.0).fit(X, y)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
# df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
# df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")
df_train_cp = pd.read_csv("Feature_matrix/df_train_cp_201808062200.csv")

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [2]:
def get_pivor_df(df, col):
    return pd.pivot_table(data= df, index="VisitNumber", fill_value=0,\
                          values="ScanCount", columns=col, aggfunc=np.sum)

def get_rid_of_zero_units(df):
    return np.where(df < 0, 0, df)

In [4]:
def get_flat_type_user(df_, col = "DepartmentDescription", is_to_get_total = False):
    """
        예:) vn_dd_more_than_one, vn_dd_one = get_flat_type_user(df_decoded) 반드시 df는 decoding 함수로 decoded된 것을 넣을 것.
    """
    df = df_.copy()
    df_count = df.groupby(["VisitNumber", col]).sum()["ScanCount"].reset_index(name="Sc_sum")
    df_count["Sc_sum"] = np.where(df_count["Sc_sum"] < 0, 0, df_count["Sc_sum"])
    df_count = df_count.dropna()
    df_count_total = df_count.groupby("VisitNumber").sum()["Sc_sum"].reset_index(name="Total")
    if is_to_get_total:
        return df_count_total

    df_merged = pd.merge(df_count, df_count_total, on="VisitNumber")
    df_merged["P"] = df_merged["Sc_sum"].div(df_merged["Total"])
    
    num_category_li = []
    for i, vn in enumerate(df_merged.VisitNumber.unique()):
        tmp_df = df_merged[df_merged["VisitNumber"] == vn]
        if tmp_df.Total.values[0] == 0:
            num_category_li.append(0)
        else:
            num_of_category = len(tmp_df[col].unique())
            num_category_li.append(num_of_category)
        if i % 5000 == 0:
            print(str(i) + "까지 진행됨.")
    
    return num_category_li

In [5]:
df_decoded = du.decodeStuffNeedsToBeDecoded(df_train)

In [6]:
len(df_decoded.Item_nbr.unique())

10007

In [7]:
df_train_item = get_pivor_df(df_decoded, "Item_nbr")

In [8]:
df_train_item = df_train_item.reset_index()

In [9]:
cols_do_not_need = ['VisitNumber', 'Return', 'TripType', 'Monday', 'Tuesday', 'Wednesday', 'Thursday',
       'Friday', 'Saturday', 'Sunday']

In [10]:
tmp = df_decoded.groupby(["VisitNumber", "DepartmentDescription"]).agg({"ScanCount" : np.sum}).reset_index()

In [11]:
tmp.groupby(["VisitNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

,VisitNumber,DepartmentDescription,ScanCount
0,5,FINANCIAL SERVICES,-1
1,7,PERSONAL CARE,1
2,7,SHOES,1
3,8,DAIRY,2
4,8,DSD GROCERY,1
5,8,HOUSEHOLD CHEMICALS/SUPP,1
6,8,MEAT - FRESH & FROZEN,1
7,8,NULL,1
8,8,PAINT AND ACCESSORIES,18
9,8,PETS AND SUPPLIES,4


In [12]:
scan_total = tmp.groupby("VisitNumber").sum()["ScanCount"].reset_index(name='ScanCount')

In [13]:
sc_li = scan_total.ScanCount.values

In [14]:
try_new_stuff = df_decoded.groupby(["Company", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [15]:
try_new_stuff_1 = try_new_stuff.groupby("Company").size().reset_index(name="Count")

In [16]:
cp_li = try_new_stuff_1[try_new_stuff_1.Count == 1].Company.unique()

In [17]:
len(cp_li)

4806

In [18]:
try_new_stuff = df_decoded.groupby(["FinelineNumber", "DepartmentDescription"]).sum()["ScanCount"].reset_index()

In [19]:
try_new_stuff_1 = try_new_stuff.groupby("FinelineNumber").size().reset_index(name="Count")

In [20]:
fl_li = try_new_stuff_1[try_new_stuff_1.Count == 1].FinelineNumber.unique()

In [21]:
len(fl_li)

2695

In [22]:
af.sendSlackDm(slack_url, "Done")

In [29]:
fl_li = fl_li.astype(float)

In [30]:
fl_li = fl_li.astype(str)

In [24]:
num_fl = get_flat_type_user(df_decoded, "FinelineNumber")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [25]:
num_dd = get_flat_type_user(df_decoded)

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [26]:
num_company = get_flat_type_user(df_decoded, "Company")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [27]:
num_upc = get_flat_type_user(df_decoded, "Upc")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [28]:
num_item = get_flat_type_user(df_decoded, "Item_nbr")

0까지 진행됨.
5000까지 진행됨.
10000까지 진행됨.
15000까지 진행됨.
20000까지 진행됨.
25000까지 진행됨.
30000까지 진행됨.
35000까지 진행됨.
40000까지 진행됨.
45000까지 진행됨.
50000까지 진행됨.
55000까지 진행됨.
60000까지 진행됨.
65000까지 진행됨.
70000까지 진행됨.
75000까지 진행됨.
80000까지 진행됨.
85000까지 진행됨.
90000까지 진행됨.
95000까지 진행됨.


In [40]:
col_dd = [col for col in df_train_dd.columns if col not in cols_do_not_need]

In [41]:
df_train_dd[col_dd] = df_train_dd[col_dd].div(pd.DataFrame(sc_li, columns=["sc"])["sc"], axis = 0)

In [89]:
len(col_fl_filttered_li), len(col_item_filttered_li), len(col_cp_filttered_li)

(1359, 2678, 2870)

In [90]:
len(col_fl_filttered_li) + len(col_item_filttered_li) + len(col_cp_filttered_li)

6907

In [67]:
cp_fl = pd.concat([df_train_dd, df_train_fl[fl_li], df_train_item[col_item_filttered_li], df_train_cp[cp_li]], axis = 1)

In [68]:
len(cp_fl)

95674

In [49]:
cp_fl["N_DD"] = num_dd

In [50]:
cp_fl["N_FL"] = num_fl

In [51]:
cp_fl["N_COMPANY"] = num_company

In [52]:
cp_fl["N_ITEM"] = num_item

In [53]:
cp_fl["N_UPC"] = 0

In [54]:
col_dd = [col for col in col_dd if col != "MENS WEAR"]

In [55]:
row_count = df_decoded.groupby("VisitNumber").count()["TripType"].values

In [56]:
cp_fl["ROW_CNT"] = row_count

In [57]:
cp_fl.set_index("VisitNumber", inplace=True)

In [58]:
df_upc = df_decoded.groupby(["VisitNumber", "Upc"]).sum()["ScanCount"].reset_index(name="Sc_sum")

In [59]:
upc_vn_li = df_upc.groupby("VisitNumber").sum()["Sc_sum"].index.unique()

In [60]:
cp_fl.at[upc_vn_li, "N_UPC"] = num_upc

ValueError: Buffer has wrong number of dimensions (expected 1, got 2)

In [ ]:
cp_fl = cp_fl.reset_index()

In [ ]:
cp_fl["Scancount_total"] = sc_li

In [ ]:
cp_fl["FL_DIV_DD"] = cp_fl["N_FL"].div(cp_fl["N_DD"], axis = 0)
cp_fl["UPC_DIV_DD"] = cp_fl["N_UPC"].div(cp_fl["N_DD"], axis = 0)
cp_fl["COMPANY_DIV_DD"] = cp_fl["N_COMPANY"].div(cp_fl["N_DD"], axis = 0)

In [ ]:
cp_fl = cp_fl.fillna(0)

In [ ]:
cp_fl["MENSWEAR"] = cp_fl["MENS WEAR"] + cp_fl["MENSWEAR"]

In [ ]:
cp_fl.drop("MENS WEAR", axis = 1, inplace=True)

In [ ]:
fl_remained_li = [col for col in fl_li if col not in zero_cols]

In [66]:
# sum_li = []
# tmp_df = cp_fl[fl_remained_li]
col_item_filttered_li = []
for idx, col in enumerate(df_train_item.columns):
    sum_of_col = df_train_item[col].sum()
#     sum_li.append(sum_of_col)
    if sum_of_col > 71:
        col_item_filttered_li.append(col)

In [372]:
cp_fl[["N_DD", "N_UPC", "N_FL", "N_ITEM", "ROW_CNT", "Scancount_total"]].describe()

,N_DD,N_UPC,N_FL,N_ITEM,ROW_CNT,Scancount_total
count,95674.000000,95674.000000,95674.000000,95674.000000,95674.000000,95674.000000
mean,3.298043,6.434360,5.833351,6.404509,6.763112,7.499467
std,2.932498,8.375563,7.356918,8.217316,8.531894,10.261119
min,0.000000,0.000000,0.000000,0.000000,1.000000,-52.000000
25%,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000
50%,2.000000,3.000000,3.000000,3.000000,4.000000,4.000000
75%,5.000000,8.000000,7.000000,8.000000,8.000000,9.000000
max,26.000000,189.000000,112.000000,186.000000,209.000000,311.000000


In [78]:
sum_li = []
col_cp_filttered_li = []
for idx, col in enumerate(df_train_cp.columns):
    sum_of_col = df_train_cp[col].sum()
    sum_li.append(sum_of_col)
    if sum_of_col >= 15.0:
        col_cp_filttered_li.append(col)

In [79]:
len(col_cp_filttered_li)

2870

In [74]:
np_sum_li = np.array(sum_li)

In [75]:
np.median(np_sum_li)

15.0

In [155]:
len(col_cp_filttered_li)

812

In [217]:
cp_fl

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,95,98,N_DD,N_FL,N_COMPANY,N_UPC,Scancount_total,FL_DIV_DD,UPC_DIV_DD,COMPANY_DIV_DD
0,5,999,1,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,...,0,0,0,0,0,0,-1,0.000000,0.000000,0.000000
1,7,30,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,2,2,2,2,2,1.000000,1.000000,1.000000
2,8,26,1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,7,17,15,21,28,2.428571,3.000000,2.142857
3,9,8,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,2,3,3,3,3,1.500000,1.500000,1.500000
4,10,8,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,2,3,3,3,3,1.500000,1.500000,1.500000
5,11,35,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,3,4,4,4,4,1.333333,1.333333,1.333333
6,12,41,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,4,7,7,7,7,1.750000,1.750000,1.750000
7,15,21,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,1,6,3,6,9,6.000000,6.000000,3.000000
8,17,6,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,2,4,4,4,4,2.000000,2.000000,2.000000
9,19,42,0,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,6,8,7,6,9,1.333333,1.000000,1.166667


In [321]:
cp_fl.drop("Return", axis = 1, inplace=True)

In [146]:
len(sum_li)

4806

In [153]:
np.array(sum_li).mean()

62.96213066999584

In [ ]:
tmp_df = cp_fl[fl_li]

In [86]:
sum_li = []
col_fl_filttered_li = []
for idx, col in enumerate(tmp_df.columns):
    sum_of_col = tmp_df[col].sum()
    sum_li.append(sum_of_col)
    if sum_of_col >= 14:
        col_fl_filttered_li.append(col)    

In [84]:
np_sum_li = np.array(sorted(sum_li, reverse=True))

In [200]:
np.median(np_sum_li)

14.0

In [87]:
len(col_fl_filttered_li)

1359

In [231]:
len(col_fl_filttered_li)

1303

In [322]:
cp_fl.head()

,VisitNumber,TripType,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,95,98,N_DD,N_FL,N_COMPANY,N_UPC,Scancount_total,FL_DIV_DD,UPC_DIV_DD,COMPANY_DIV_DD
0,5,999,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,0,0,0,0,0,0,-1,0.000000,0.0,0.000000
1,7,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,2,2,2,2,2,1.000000,1.0,1.000000
2,8,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,7,17,15,21,28,2.428571,3.0,2.142857
3,9,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,2,3,3,3,3,1.500000,1.5,1.500000
4,10,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,2,3,3,3,3,1.500000,1.5,1.500000


In [58]:
from scipy.sparse import csr_matrix
import lightgbm

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [323]:
train_X , train_y = af.get_df_to_fit(cp_fl)

In [324]:
label_enc = LabelEncoder().fit(train_y)
y_labeled = label_enc.transform(train_y)

In [325]:
train_X, test_X, train_y, test_y = train_test_split(train_X, y_labeled, random_state=0)

In [326]:
train_X = train_X.astype(float)
test_X = test_X.astype(float)

In [327]:
X_train = csr_matrix(train_X.values)
X_test = csr_matrix(test_X.values)

In [328]:
dtrain = lightgbm.Dataset(X_train, label=train_y)
dtest = lightgbm.Dataset(X_test, label=test_y)

In [330]:
num_boost_round = 4000
max_delta_step=0
learning_rate=0.02

params = {'objective':'multiclass',
          'boosting_type': 'gbdt',
          'max_depth' : -1,
          'nthread': 4,
          'metric': 'multi_logloss',
          'num_class':38,
          'learning_rate':learning_rate,
          'max_delta_step':max_delta_step,
          }

# evals = [(dtrain, 'train'), (dtest, 'test')]

# %%time
# 4000, 0.008
lightgbm_model = lightgbm.train(params = params,
                                train_set = dtrain, 
                                valid_sets = [dtrain, dtest],
                                num_boost_round = num_boost_round,
                                early_stopping_rounds=10)

[1]	training's multi_logloss: 3.42366	valid_1's multi_logloss: 3.43017
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 3.26244	valid_1's multi_logloss: 3.27351
[3]	training's multi_logloss: 3.12985	valid_1's multi_logloss: 3.14469
[4]	training's multi_logloss: 3.01658	valid_1's multi_logloss: 3.03488
[5]	training's multi_logloss: 2.91769	valid_1's multi_logloss: 2.93909
[6]	training's multi_logloss: 2.8297	valid_1's multi_logloss: 2.85389
[7]	training's multi_logloss: 2.74994	valid_1's multi_logloss: 2.77678
[8]	training's multi_logloss: 2.67727	valid_1's multi_logloss: 2.70647
[9]	training's multi_logloss: 2.6105	valid_1's multi_logloss: 2.64189
[10]	training's multi_logloss: 2.54875	valid_1's multi_logloss: 2.58226
[11]	training's multi_logloss: 2.49113	valid_1's multi_logloss: 2.52655
[12]	training's multi_logloss: 2.43714	valid_1's multi_logloss: 2.47455
[13]	training's multi_logloss: 2.3865	valid_1's multi_logloss: 2.42576
[14]	training'

[115]	training's multi_logloss: 0.926766	valid_1's multi_logloss: 1.04731
[116]	training's multi_logloss: 0.922337	valid_1's multi_logloss: 1.04343
[117]	training's multi_logloss: 0.917967	valid_1's multi_logloss: 1.03961
[118]	training's multi_logloss: 0.913677	valid_1's multi_logloss: 1.03592
[119]	training's multi_logloss: 0.909427	valid_1's multi_logloss: 1.03223
[120]	training's multi_logloss: 0.905203	valid_1's multi_logloss: 1.02856
[121]	training's multi_logloss: 0.901077	valid_1's multi_logloss: 1.02505
[122]	training's multi_logloss: 0.89692	valid_1's multi_logloss: 1.02142
[123]	training's multi_logloss: 0.892858	valid_1's multi_logloss: 1.01791
[124]	training's multi_logloss: 0.888836	valid_1's multi_logloss: 1.01449
[125]	training's multi_logloss: 0.884895	valid_1's multi_logloss: 1.01114
[126]	training's multi_logloss: 0.881023	valid_1's multi_logloss: 1.00782
[127]	training's multi_logloss: 0.877205	valid_1's multi_logloss: 1.00453
[128]	training's multi_logloss: 0.87342

[225]	training's multi_logloss: 0.653006	valid_1's multi_logloss: 0.829046
[226]	training's multi_logloss: 0.65157	valid_1's multi_logloss: 0.82812
[227]	training's multi_logloss: 0.650212	valid_1's multi_logloss: 0.827244
[228]	training's multi_logloss: 0.648855	valid_1's multi_logloss: 0.826345
[229]	training's multi_logloss: 0.647476	valid_1's multi_logloss: 0.825468
[230]	training's multi_logloss: 0.646139	valid_1's multi_logloss: 0.824593
[231]	training's multi_logloss: 0.644817	valid_1's multi_logloss: 0.823711
[232]	training's multi_logloss: 0.643461	valid_1's multi_logloss: 0.822833
[233]	training's multi_logloss: 0.642159	valid_1's multi_logloss: 0.821982
[234]	training's multi_logloss: 0.640842	valid_1's multi_logloss: 0.821142
[235]	training's multi_logloss: 0.639525	valid_1's multi_logloss: 0.820283
[236]	training's multi_logloss: 0.638223	valid_1's multi_logloss: 0.819483
[237]	training's multi_logloss: 0.636941	valid_1's multi_logloss: 0.818695
[238]	training's multi_logl

[335]	training's multi_logloss: 0.541983	valid_1's multi_logloss: 0.768676
[336]	training's multi_logloss: 0.541244	valid_1's multi_logloss: 0.768369
[337]	training's multi_logloss: 0.540494	valid_1's multi_logloss: 0.76806
[338]	training's multi_logloss: 0.539741	valid_1's multi_logloss: 0.767736
[339]	training's multi_logloss: 0.538999	valid_1's multi_logloss: 0.767447
[340]	training's multi_logloss: 0.538281	valid_1's multi_logloss: 0.767163
[341]	training's multi_logloss: 0.537538	valid_1's multi_logloss: 0.766869
[342]	training's multi_logloss: 0.536804	valid_1's multi_logloss: 0.766578
[343]	training's multi_logloss: 0.536062	valid_1's multi_logloss: 0.766254
[344]	training's multi_logloss: 0.535361	valid_1's multi_logloss: 0.765968
[345]	training's multi_logloss: 0.534637	valid_1's multi_logloss: 0.7657
[346]	training's multi_logloss: 0.533927	valid_1's multi_logloss: 0.76542
[347]	training's multi_logloss: 0.533224	valid_1's multi_logloss: 0.765119
[348]	training's multi_loglos

[445]	training's multi_logloss: 0.472773	valid_1's multi_logloss: 0.745248
[446]	training's multi_logloss: 0.472243	valid_1's multi_logloss: 0.745115
[447]	training's multi_logloss: 0.471713	valid_1's multi_logloss: 0.744971
[448]	training's multi_logloss: 0.471193	valid_1's multi_logloss: 0.744854
[449]	training's multi_logloss: 0.470652	valid_1's multi_logloss: 0.74474
[450]	training's multi_logloss: 0.470118	valid_1's multi_logloss: 0.744586
[451]	training's multi_logloss: 0.469574	valid_1's multi_logloss: 0.744457
[452]	training's multi_logloss: 0.469038	valid_1's multi_logloss: 0.7443
[453]	training's multi_logloss: 0.468514	valid_1's multi_logloss: 0.744153
[454]	training's multi_logloss: 0.46801	valid_1's multi_logloss: 0.744028
[455]	training's multi_logloss: 0.467487	valid_1's multi_logloss: 0.743877
[456]	training's multi_logloss: 0.466961	valid_1's multi_logloss: 0.743744
[457]	training's multi_logloss: 0.466443	valid_1's multi_logloss: 0.743617
[458]	training's multi_loglos

[555]	training's multi_logloss: 0.4218	valid_1's multi_logloss: 0.73373
[556]	training's multi_logloss: 0.421392	valid_1's multi_logloss: 0.73367
[557]	training's multi_logloss: 0.420976	valid_1's multi_logloss: 0.733597
[558]	training's multi_logloss: 0.420572	valid_1's multi_logloss: 0.733506
[559]	training's multi_logloss: 0.420169	valid_1's multi_logloss: 0.733443
[560]	training's multi_logloss: 0.419762	valid_1's multi_logloss: 0.733388
[561]	training's multi_logloss: 0.41935	valid_1's multi_logloss: 0.733331
[562]	training's multi_logloss: 0.418939	valid_1's multi_logloss: 0.733252
[563]	training's multi_logloss: 0.418533	valid_1's multi_logloss: 0.733175
[564]	training's multi_logloss: 0.418143	valid_1's multi_logloss: 0.733088
[565]	training's multi_logloss: 0.41774	valid_1's multi_logloss: 0.732991
[566]	training's multi_logloss: 0.417337	valid_1's multi_logloss: 0.732933
[567]	training's multi_logloss: 0.416934	valid_1's multi_logloss: 0.732841
[568]	training's multi_logloss:

[665]	training's multi_logloss: 0.381961	valid_1's multi_logloss: 0.727632
[666]	training's multi_logloss: 0.381616	valid_1's multi_logloss: 0.727576
[667]	training's multi_logloss: 0.38131	valid_1's multi_logloss: 0.727499
[668]	training's multi_logloss: 0.380995	valid_1's multi_logloss: 0.727444
[669]	training's multi_logloss: 0.380677	valid_1's multi_logloss: 0.727381
[670]	training's multi_logloss: 0.380353	valid_1's multi_logloss: 0.727366
[671]	training's multi_logloss: 0.380025	valid_1's multi_logloss: 0.727338
[672]	training's multi_logloss: 0.379711	valid_1's multi_logloss: 0.727299
[673]	training's multi_logloss: 0.379388	valid_1's multi_logloss: 0.727251
[674]	training's multi_logloss: 0.37907	valid_1's multi_logloss: 0.727213
[675]	training's multi_logloss: 0.378754	valid_1's multi_logloss: 0.727184
[676]	training's multi_logloss: 0.378442	valid_1's multi_logloss: 0.727161
[677]	training's multi_logloss: 0.378117	valid_1's multi_logloss: 0.727139
[678]	training's multi_logl

[775]	training's multi_logloss: 0.349652	valid_1's multi_logloss: 0.72433
[776]	training's multi_logloss: 0.349389	valid_1's multi_logloss: 0.724329
[777]	training's multi_logloss: 0.349117	valid_1's multi_logloss: 0.724312
[778]	training's multi_logloss: 0.348849	valid_1's multi_logloss: 0.724301
[779]	training's multi_logloss: 0.348584	valid_1's multi_logloss: 0.72428
[780]	training's multi_logloss: 0.348318	valid_1's multi_logloss: 0.724253
[781]	training's multi_logloss: 0.348056	valid_1's multi_logloss: 0.724217
[782]	training's multi_logloss: 0.347792	valid_1's multi_logloss: 0.72421
[783]	training's multi_logloss: 0.347533	valid_1's multi_logloss: 0.724196
[784]	training's multi_logloss: 0.347252	valid_1's multi_logloss: 0.724187
[785]	training's multi_logloss: 0.346989	valid_1's multi_logloss: 0.724171
[786]	training's multi_logloss: 0.346721	valid_1's multi_logloss: 0.724162
[787]	training's multi_logloss: 0.346447	valid_1's multi_logloss: 0.724155
[788]	training's multi_loglo

# fl(>median), company(>median), item_nbr(>mean) dummies 6425 컬럼. 0.723162

# fl, company dummie들의 컬럼으로 Sum한 값을 확인해서 둘다 median 값 이상으로 팔린 아이템만 남긴 feature로 돌렸다. 컬럼은 3748 점수는 0.756773

In [226]:
num_boost_round = 4000
max_delta_step=0
learning_rate=0.02

params = {'objective':'multiclass',
          'boosting_type': 'gbdt',
          'max_depth' : -1,
          'nthread': 4,
          'metric': 'multi_logloss',
          'max_delta_step': 3, 
          'num_class':38,
          'learning_rate':learning_rate,
          'max_delta_step':max_delta_step,
          }

# evals = [(dtrain, 'train'), (dtest, 'test')]

# %%time
# 4000, 0.008
lightgbm_model = lightgbm.train(params = params,
                                train_set = dtrain, 
                                valid_sets = [dtrain, dtest],
                                num_boost_round = num_boost_round,
                                early_stopping_rounds=10)

[1]	training's multi_logloss: 3.42656	valid_1's multi_logloss: 3.43305
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 3.26719	valid_1's multi_logloss: 3.27824
[3]	training's multi_logloss: 3.13593	valid_1's multi_logloss: 3.15062
[4]	training's multi_logloss: 3.02374	valid_1's multi_logloss: 3.04172
[5]	training's multi_logloss: 2.92567	valid_1's multi_logloss: 2.94663
[6]	training's multi_logloss: 2.83838	valid_1's multi_logloss: 2.86201
[7]	training's multi_logloss: 2.75914	valid_1's multi_logloss: 2.78528
[8]	training's multi_logloss: 2.6869	valid_1's multi_logloss: 2.71532
[9]	training's multi_logloss: 2.62058	valid_1's multi_logloss: 2.65115
[10]	training's multi_logloss: 2.5592	valid_1's multi_logloss: 2.59203
[11]	training's multi_logloss: 2.5021	valid_1's multi_logloss: 2.53673
[12]	training's multi_logloss: 2.44853	valid_1's multi_logloss: 2.48512
[13]	training's multi_logloss: 2.39816	valid_1's multi_logloss: 2.43668
[14]	training'

[115]	training's multi_logloss: 0.949936	valid_1's multi_logloss: 1.06851
[116]	training's multi_logloss: 0.945594	valid_1's multi_logloss: 1.06474
[117]	training's multi_logloss: 0.94125	valid_1's multi_logloss: 1.06094
[118]	training's multi_logloss: 0.937037	valid_1's multi_logloss: 1.05729
[119]	training's multi_logloss: 0.932854	valid_1's multi_logloss: 1.05367
[120]	training's multi_logloss: 0.928741	valid_1's multi_logloss: 1.05013
[121]	training's multi_logloss: 0.924665	valid_1's multi_logloss: 1.04661
[122]	training's multi_logloss: 0.920642	valid_1's multi_logloss: 1.04317
[123]	training's multi_logloss: 0.916685	valid_1's multi_logloss: 1.03974
[124]	training's multi_logloss: 0.912806	valid_1's multi_logloss: 1.03639
[125]	training's multi_logloss: 0.908961	valid_1's multi_logloss: 1.0331
[126]	training's multi_logloss: 0.905178	valid_1's multi_logloss: 1.02985
[127]	training's multi_logloss: 0.90137	valid_1's multi_logloss: 1.02664
[128]	training's multi_logloss: 0.897668	

[225]	training's multi_logloss: 0.683836	valid_1's multi_logloss: 0.856278
[226]	training's multi_logloss: 0.682519	valid_1's multi_logloss: 0.855447
[227]	training's multi_logloss: 0.681212	valid_1's multi_logloss: 0.854638
[228]	training's multi_logloss: 0.679923	valid_1's multi_logloss: 0.853804
[229]	training's multi_logloss: 0.678612	valid_1's multi_logloss: 0.852938
[230]	training's multi_logloss: 0.677315	valid_1's multi_logloss: 0.852145
[231]	training's multi_logloss: 0.676056	valid_1's multi_logloss: 0.851357
[232]	training's multi_logloss: 0.674766	valid_1's multi_logloss: 0.850575
[233]	training's multi_logloss: 0.673539	valid_1's multi_logloss: 0.849806
[234]	training's multi_logloss: 0.672311	valid_1's multi_logloss: 0.849043
[235]	training's multi_logloss: 0.671044	valid_1's multi_logloss: 0.84827
[236]	training's multi_logloss: 0.669818	valid_1's multi_logloss: 0.847509
[237]	training's multi_logloss: 0.668607	valid_1's multi_logloss: 0.846789
[238]	training's multi_log

[335]	training's multi_logloss: 0.579725	valid_1's multi_logloss: 0.800407
[336]	training's multi_logloss: 0.579016	valid_1's multi_logloss: 0.800111
[337]	training's multi_logloss: 0.578353	valid_1's multi_logloss: 0.799847
[338]	training's multi_logloss: 0.577666	valid_1's multi_logloss: 0.799572
[339]	training's multi_logloss: 0.576983	valid_1's multi_logloss: 0.799305
[340]	training's multi_logloss: 0.576324	valid_1's multi_logloss: 0.799013
[341]	training's multi_logloss: 0.575644	valid_1's multi_logloss: 0.798756
[342]	training's multi_logloss: 0.57495	valid_1's multi_logloss: 0.798489
[343]	training's multi_logloss: 0.574284	valid_1's multi_logloss: 0.798209
[344]	training's multi_logloss: 0.573611	valid_1's multi_logloss: 0.797945
[345]	training's multi_logloss: 0.572947	valid_1's multi_logloss: 0.797671
[346]	training's multi_logloss: 0.572302	valid_1's multi_logloss: 0.797416
[347]	training's multi_logloss: 0.571648	valid_1's multi_logloss: 0.797132
[348]	training's multi_log

[445]	training's multi_logloss: 0.517159	valid_1's multi_logloss: 0.78083
[446]	training's multi_logloss: 0.51668	valid_1's multi_logloss: 0.780717
[447]	training's multi_logloss: 0.516218	valid_1's multi_logloss: 0.780599
[448]	training's multi_logloss: 0.515743	valid_1's multi_logloss: 0.780528
[449]	training's multi_logloss: 0.515282	valid_1's multi_logloss: 0.780416
[450]	training's multi_logloss: 0.51481	valid_1's multi_logloss: 0.78032
[451]	training's multi_logloss: 0.514359	valid_1's multi_logloss: 0.780226
[452]	training's multi_logloss: 0.513894	valid_1's multi_logloss: 0.780134
[453]	training's multi_logloss: 0.513439	valid_1's multi_logloss: 0.780033
[454]	training's multi_logloss: 0.512972	valid_1's multi_logloss: 0.779957
[455]	training's multi_logloss: 0.512528	valid_1's multi_logloss: 0.779872
[456]	training's multi_logloss: 0.512079	valid_1's multi_logloss: 0.779789
[457]	training's multi_logloss: 0.511616	valid_1's multi_logloss: 0.779711
[458]	training's multi_loglos

[555]	training's multi_logloss: 0.472012	valid_1's multi_logloss: 0.773671
[556]	training's multi_logloss: 0.471664	valid_1's multi_logloss: 0.7736
[557]	training's multi_logloss: 0.471309	valid_1's multi_logloss: 0.773563
[558]	training's multi_logloss: 0.470956	valid_1's multi_logloss: 0.77353
[559]	training's multi_logloss: 0.470589	valid_1's multi_logloss: 0.773511
[560]	training's multi_logloss: 0.470214	valid_1's multi_logloss: 0.77346
[561]	training's multi_logloss: 0.469865	valid_1's multi_logloss: 0.773429
[562]	training's multi_logloss: 0.469508	valid_1's multi_logloss: 0.773373
[563]	training's multi_logloss: 0.469139	valid_1's multi_logloss: 0.773351
[564]	training's multi_logloss: 0.468784	valid_1's multi_logloss: 0.773342
[565]	training's multi_logloss: 0.468433	valid_1's multi_logloss: 0.77329
[566]	training's multi_logloss: 0.468085	valid_1's multi_logloss: 0.773265
[567]	training's multi_logloss: 0.467727	valid_1's multi_logloss: 0.77324
[568]	training's multi_logloss:

[665]	training's multi_logloss: 0.436445	valid_1's multi_logloss: 0.771343
[666]	training's multi_logloss: 0.436155	valid_1's multi_logloss: 0.771334
[667]	training's multi_logloss: 0.43587	valid_1's multi_logloss: 0.771307
[668]	training's multi_logloss: 0.435574	valid_1's multi_logloss: 0.771288
[669]	training's multi_logloss: 0.43528	valid_1's multi_logloss: 0.771267
[670]	training's multi_logloss: 0.434989	valid_1's multi_logloss: 0.771249
[671]	training's multi_logloss: 0.434692	valid_1's multi_logloss: 0.771244
[672]	training's multi_logloss: 0.434394	valid_1's multi_logloss: 0.771224
[673]	training's multi_logloss: 0.434126	valid_1's multi_logloss: 0.771213
[674]	training's multi_logloss: 0.433838	valid_1's multi_logloss: 0.771211
[675]	training's multi_logloss: 0.433543	valid_1's multi_logloss: 0.771212
[676]	training's multi_logloss: 0.433261	valid_1's multi_logloss: 0.771208
[677]	training's multi_logloss: 0.432969	valid_1's multi_logloss: 0.771225
[678]	training's multi_logl

In [251]:
X_train

<71755x1374 sparse matrix of type '<class 'numpy.float64'>'
	with 1059322 stored elements in Compressed Sparse Row format>

# fl, company dummie들의 컬럼으로 Sum한 값을 확인해서 둘다 mean 값 이상으로 팔린 아이템만 남긴 feature로 돌렸다. 컬럼은 1374 점수는 0.771142

In [185]:
num_boost_round = 4000
max_delta_step=0
learning_rate=0.02

params = {'objective':'multiclass',
          'boosting_type': 'gbdt',
          'max_depth' : -1,
          'nthread': 4,
          'metric': 'multi_logloss',
          'max_delta_step': 3, 
          'num_class':38,
          'learning_rate':learning_rate,
          'max_delta_step':max_delta_step,
          }

# evals = [(dtrain, 'train'), (dtest, 'test')]

# %%time
# 4000, 0.008
lightgbm_model = lightgbm.train(params = params,
                                train_set = dtrain, 
                                valid_sets = [dtrain, dtest],
                                num_boost_round = num_boost_round,
                                early_stopping_rounds=10)

[1]	training's multi_logloss: 3.42595	valid_1's multi_logloss: 3.43262
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 3.2662	valid_1's multi_logloss: 3.27737
[3]	training's multi_logloss: 3.13468	valid_1's multi_logloss: 3.14957
[4]	training's multi_logloss: 3.02222	valid_1's multi_logloss: 3.04044
[5]	training's multi_logloss: 2.92399	valid_1's multi_logloss: 2.94528
[6]	training's multi_logloss: 2.83658	valid_1's multi_logloss: 2.86055
[7]	training's multi_logloss: 2.75722	valid_1's multi_logloss: 2.78367
[8]	training's multi_logloss: 2.68489	valid_1's multi_logloss: 2.7137
[9]	training's multi_logloss: 2.61845	valid_1's multi_logloss: 2.64948
[10]	training's multi_logloss: 2.55698	valid_1's multi_logloss: 2.59027
[11]	training's multi_logloss: 2.49978	valid_1's multi_logloss: 2.53488
[12]	training's multi_logloss: 2.44612	valid_1's multi_logloss: 2.48318
[13]	training's multi_logloss: 2.39571	valid_1's multi_logloss: 2.43481
[14]	training

[115]	training's multi_logloss: 0.945275	valid_1's multi_logloss: 1.06397
[116]	training's multi_logloss: 0.940863	valid_1's multi_logloss: 1.06011
[117]	training's multi_logloss: 0.936589	valid_1's multi_logloss: 1.05631
[118]	training's multi_logloss: 0.932323	valid_1's multi_logloss: 1.05258
[119]	training's multi_logloss: 0.92816	valid_1's multi_logloss: 1.04894
[120]	training's multi_logloss: 0.923983	valid_1's multi_logloss: 1.04531
[121]	training's multi_logloss: 0.919909	valid_1's multi_logloss: 1.04181
[122]	training's multi_logloss: 0.915917	valid_1's multi_logloss: 1.03834
[123]	training's multi_logloss: 0.91197	valid_1's multi_logloss: 1.03496
[124]	training's multi_logloss: 0.908037	valid_1's multi_logloss: 1.03156
[125]	training's multi_logloss: 0.90418	valid_1's multi_logloss: 1.02822
[126]	training's multi_logloss: 0.900388	valid_1's multi_logloss: 1.02496
[127]	training's multi_logloss: 0.89664	valid_1's multi_logloss: 1.02173
[128]	training's multi_logloss: 0.892916	v

[225]	training's multi_logloss: 0.677872	valid_1's multi_logloss: 0.849628
[226]	training's multi_logloss: 0.676547	valid_1's multi_logloss: 0.848757
[227]	training's multi_logloss: 0.67522	valid_1's multi_logloss: 0.847889
[228]	training's multi_logloss: 0.673903	valid_1's multi_logloss: 0.847028
[229]	training's multi_logloss: 0.672611	valid_1's multi_logloss: 0.846188
[230]	training's multi_logloss: 0.671313	valid_1's multi_logloss: 0.845368
[231]	training's multi_logloss: 0.670042	valid_1's multi_logloss: 0.84453
[232]	training's multi_logloss: 0.668757	valid_1's multi_logloss: 0.843701
[233]	training's multi_logloss: 0.667476	valid_1's multi_logloss: 0.842888
[234]	training's multi_logloss: 0.666222	valid_1's multi_logloss: 0.842076
[235]	training's multi_logloss: 0.664975	valid_1's multi_logloss: 0.841283
[236]	training's multi_logloss: 0.663723	valid_1's multi_logloss: 0.840474
[237]	training's multi_logloss: 0.662497	valid_1's multi_logloss: 0.839683
[238]	training's multi_logl

[335]	training's multi_logloss: 0.573016	valid_1's multi_logloss: 0.792797
[336]	training's multi_logloss: 0.572335	valid_1's multi_logloss: 0.792537
[337]	training's multi_logloss: 0.571635	valid_1's multi_logloss: 0.792248
[338]	training's multi_logloss: 0.57095	valid_1's multi_logloss: 0.791946
[339]	training's multi_logloss: 0.570275	valid_1's multi_logloss: 0.791672
[340]	training's multi_logloss: 0.569605	valid_1's multi_logloss: 0.791389
[341]	training's multi_logloss: 0.568919	valid_1's multi_logloss: 0.791107
[342]	training's multi_logloss: 0.568232	valid_1's multi_logloss: 0.790845
[343]	training's multi_logloss: 0.567577	valid_1's multi_logloss: 0.790563
[344]	training's multi_logloss: 0.5669	valid_1's multi_logloss: 0.790311
[345]	training's multi_logloss: 0.566224	valid_1's multi_logloss: 0.790045
[346]	training's multi_logloss: 0.565564	valid_1's multi_logloss: 0.789745
[347]	training's multi_logloss: 0.564909	valid_1's multi_logloss: 0.789496
[348]	training's multi_loglo

[445]	training's multi_logloss: 0.509746	valid_1's multi_logloss: 0.772335
[446]	training's multi_logloss: 0.509277	valid_1's multi_logloss: 0.772213
[447]	training's multi_logloss: 0.508802	valid_1's multi_logloss: 0.772103
[448]	training's multi_logloss: 0.508312	valid_1's multi_logloss: 0.771979
[449]	training's multi_logloss: 0.507844	valid_1's multi_logloss: 0.771883
[450]	training's multi_logloss: 0.507382	valid_1's multi_logloss: 0.771778
[451]	training's multi_logloss: 0.506915	valid_1's multi_logloss: 0.771682
[452]	training's multi_logloss: 0.506462	valid_1's multi_logloss: 0.771569
[453]	training's multi_logloss: 0.505998	valid_1's multi_logloss: 0.771457
[454]	training's multi_logloss: 0.505531	valid_1's multi_logloss: 0.771357
[455]	training's multi_logloss: 0.505072	valid_1's multi_logloss: 0.771231
[456]	training's multi_logloss: 0.504605	valid_1's multi_logloss: 0.771131
[457]	training's multi_logloss: 0.504157	valid_1's multi_logloss: 0.771055
[458]	training's multi_lo

[555]	training's multi_logloss: 0.464337	valid_1's multi_logloss: 0.764746
[556]	training's multi_logloss: 0.463967	valid_1's multi_logloss: 0.764691
[557]	training's multi_logloss: 0.463588	valid_1's multi_logloss: 0.764656
[558]	training's multi_logloss: 0.46323	valid_1's multi_logloss: 0.764613
[559]	training's multi_logloss: 0.462874	valid_1's multi_logloss: 0.764583
[560]	training's multi_logloss: 0.462517	valid_1's multi_logloss: 0.76452
[561]	training's multi_logloss: 0.462163	valid_1's multi_logloss: 0.764478
[562]	training's multi_logloss: 0.46181	valid_1's multi_logloss: 0.764459
[563]	training's multi_logloss: 0.46146	valid_1's multi_logloss: 0.764397
[564]	training's multi_logloss: 0.461113	valid_1's multi_logloss: 0.764333
[565]	training's multi_logloss: 0.460761	valid_1's multi_logloss: 0.764302
[566]	training's multi_logloss: 0.460414	valid_1's multi_logloss: 0.76428
[567]	training's multi_logloss: 0.460055	valid_1's multi_logloss: 0.764222
[568]	training's multi_logloss

[665]	training's multi_logloss: 0.428552	valid_1's multi_logloss: 0.761355
[666]	training's multi_logloss: 0.428262	valid_1's multi_logloss: 0.761324
[667]	training's multi_logloss: 0.427967	valid_1's multi_logloss: 0.761313
[668]	training's multi_logloss: 0.427677	valid_1's multi_logloss: 0.761283
[669]	training's multi_logloss: 0.427378	valid_1's multi_logloss: 0.761269
[670]	training's multi_logloss: 0.427093	valid_1's multi_logloss: 0.761263
[671]	training's multi_logloss: 0.426812	valid_1's multi_logloss: 0.761254
[672]	training's multi_logloss: 0.426507	valid_1's multi_logloss: 0.761234
[673]	training's multi_logloss: 0.426223	valid_1's multi_logloss: 0.761237
[674]	training's multi_logloss: 0.42594	valid_1's multi_logloss: 0.761216
[675]	training's multi_logloss: 0.42565	valid_1's multi_logloss: 0.761208
[676]	training's multi_logloss: 0.425373	valid_1's multi_logloss: 0.761203
[677]	training's multi_logloss: 0.42508	valid_1's multi_logloss: 0.761195
[678]	training's multi_loglo

# fl, company dummie들의 컬럼으로 Sum한 값을 확인해서 fl은 14(중앙값)이상, (company는) mean 값 이상으로 팔린 아이템만 남긴 feature로 돌렸다. 컬럼은 2236 점수는 0.760797

In [186]:
af.sendSlackDm(slack_url, "Done")